In [1]:
# bibliotecas
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from tensorflow.keras.applications import Xception
from tensorflow.keras import layers, models

from sklearn.metrics import classification_report, confusion_matrix, f1_score, roc_auc_score

### Carregamento e Pré-Processamento do Dataset

In [2]:
from utils.data_utils_keras import load_and_preprocess_data_tf, ModelType

batch_size = 32
image_size = (299, 299)
train_ds, test_ds = load_and_preprocess_data_tf(resize_to=image_size,
                                                batch_size=batch_size,
                                                model_type=ModelType.XCEPTION)

Found 100000 files belonging to 2 classes.
Found 20000 files belonging to 2 classes.


### Divisão Holdout

In [3]:
total_batches = tf.data.experimental.cardinality(train_ds).numpy()
total_samples = total_batches * batch_size

In [4]:
# Define proporção de treino (70%) e validação (30%) usando o número de batches
train_batches = int(total_batches * 0.7)
val_batches = total_batches - train_batches

In [5]:
# Embaralha os batches do dataset
train_ds_shuffled = train_ds.shuffle(buffer_size=1000, seed=42, reshuffle_each_iteration=False)

In [6]:
# Divisão trenio e validação
train_ds = train_ds_shuffled.take(train_batches)
val_ds = train_ds_shuffled.skip(train_batches)

train_ds = train_ds.prefetch(tf.data.AUTOTUNE)
val_ds = val_ds.prefetch(tf.data.AUTOTUNE)

In [7]:
print(f"Batches de treino: {train_batches}")
print(f"Batches de validação: {val_batches}")
print(f"Aproximadamente {train_batches * batch_size} amostras de treino")
print(f"Aproximadamente {val_batches * batch_size} amostras de validação")

Batches de treino: 2187
Batches de validação: 938
Aproximadamente 69984 amostras de treino
Aproximadamente 30016 amostras de validação


## Treino

### Modelo

**Xception**

A Xception Architecture é uma arquitetura de rede neural convolucional que foi introduzida em 2016 por François Chollet, o criador da biblioteca de deep learning Keras. Essa arquitetura é conhecida por sua eficiência e desempenho superior em comparação com outras arquiteturas convolucionais tradicionais. A Xception Architecture é baseada no conceito de depthwise separable convolutions, que consiste em dividir a convolução em duas etapas separadas: uma etapa para lidar com a convolução espacial e outra para lidar com a convolução em profundidade. Isso permite que a rede aprenda representações mais ricas e complexas, resultando em um melhor desempenho em tarefas de visão computacional.

Diferentemente de arquiteturas como a Inception, que combinam múltiplos tipos de convolução em paralelo, a Xception simplifica esse processo ao aplicar uma convolução depthwise (que opera separadamente em cada canal de entrada) seguida por uma convolução pointwise (1x1), que combina os canais. Essa separação permite um modelo mais leve e profundo, com menor número de parâmetros e operações, sem comprometer a capacidade de aprendizado. A arquitetura é composta por três partes principais: uma entrada com camadas convolucionais iniciais, um corpo formado por 14 módulos principais baseados em depthwise separable convolutions com conexões residuais, e uma saída com pooling global e uma camada densa para classificação.

Fonte: https://iatracker.com.br/glossario/o-que-e-xception-architecture/

In [ ]:
base_model = Xception(weights='imagenet', include_top=False, input_shape=(*image_size, 3))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    batch_size=batch_size,
    verbose=1
)

In [ ]:
y_pred_prob = model.predict(test_ds)
y_pred = (y_pred_prob > 0.5).astype(int).ravel()

# Extrair labels verdadeiros do dataset de teste
y_test = []
for _, labels in test_ds:
    y_test.extend(labels.numpy())
y_test = np.array(y_test)

# Flatten y_pred_prob para garantir formato correto
y_pred_prob = y_pred_prob.ravel()

cm = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(5, 4))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Real', 'Fake'], yticklabels=['Real', 'Fake'])
plt.xlabel("Predito")
plt.ylabel("Real")
plt.title("Matriz de Confusão")
plt.show()

print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred, target_names=["Real", "Fake"]))

roc_auc = roc_auc_score(y_test, y_pred_prob)
print(f"AUC (ROC): {roc_auc:.4f}")

plt.figure(figsize=(8, 4))

# Acurácia
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Treino')
plt.plot(history.history['val_accuracy'], label='Validação')
plt.title("Curva de Aprendizado - Acurácia")
plt.xlabel("Época")
plt.ylabel("Acurácia")
plt.legend()
plt.grid()

# Perda
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Treino')
plt.plot(history.history['val_loss'], label='Validação')
plt.title("Curva de Aprendizado - Perda")
plt.xlabel("Época")
plt.ylabel("Perda (Loss)")
plt.legend()
plt.grid()

plt.show()